# This code calculates the total ice concentration for Coronation Gulf from the weekly CIS regional charts.

Note: the E00 charts have been converted to shapefile to the SIGRID 2004 format.

In [ ]:
from os import listdir
import numpy as np
import geopandas as gp
from datetime import datetime
import pandas as pd

Finds all the SIGRID shapefiles<br>
Note: CIS climatology files can be downloaded here: https://iceweb1.cis.ec.gc.ca/Archive/page1.xhtml?lang=en

In [ ]:
files = [f for f in listdir(r'../../data/SIGRID') if f.endswith('.zip')]

Reads the files, clips the Coronation Gulf marine region and calculates the total SIC using GeoPandas

In [ ]:
cg = gp.read_file(r'../../data/Coronation_simple.shp')

df = pd.DataFrame({'Date':[],'IceArea(m2)':[],'TotalArea(m2)':[],'SIC':[]})

for file in files:
    
    date = datetime.strptime(file[:-4], '%Y%m%d')
    print(date)
    
    chart = gp.read_file(r'zip://Data/SIGRID/' + file)
    if 'CT' in chart.keys():
        chart_clip = gp.overlay(chart.dropna(subset=['CT'])[['AREA','CT','geometry']],
                                cg.to_crs(chart.crs)[['NAME','geometry']],
                                how='intersection')
        chart_clip.CT=chart_clip.CT.astype('float32')

        icearea = np.sum(np.ceil(chart_clip.CT[(chart_clip.CT>=10) & (chart_clip.CT<=92)]/10)/10*
                         chart_clip[(chart_clip.CT>=10) & (chart_clip.CT<=92)].geometry.area)
        totarea = chart_clip.geometry.area.sum()

        df = df.append(pd.DataFrame({'Date':[date],'IceArea(m2)':[icearea],'TotalArea(m2)':[totarea],
                                     'SIC':[icearea/totarea*100]}))
    else:
        chart_clip = gp.overlay(chart.dropna(subset=['SGD_CT'])[['AREA','SGD_CT','geometry']],
                                cg.to_crs(chart.crs)[['NAME','geometry']],
                                how='intersection')
        chart_clip.SGD_CT=chart_clip.SGD_CT.astype('float32')

        icearea = np.sum(np.ceil(chart_clip.SGD_CT[(chart_clip.SGD_CT>=10) & (chart_clip.SGD_CT<=92)]/10)/10*
                         chart_clip[(chart_clip.SGD_CT>=10) & (chart_clip.SGD_CT<=92)].geometry.area)
        totarea = np.sum(chart_clip.geometry.area.sum())

        df = df.append(pd.DataFrame({'Date':[date],'IceArea(m2)':[icearea],'TotalArea(m2)':[totarea],
                                     'SIC':[icearea/totarea*100]}))
    

Saves the dataframe to a pickle

In [ ]:
df.to_pickle(r'../../data/WeeklySIC_CIS.pkl')